In [1]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("PiC/phrase_similarity")

/home/hari/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Generating train split:   0%|          | 0/7004 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Features that could be important:

- Greedy word-wise similarity of phrase
- Lesk overlap with content words in context sentence


In [2]:
import pandas as pd


train = pd.DataFrame(dataset['train'])
test = pd.DataFrame(dataset['test'])
val = pd.DataFrame(dataset['validation'])

In [3]:
train

,phrase1,phrase2,sentence1,sentence2,label,idx
0,newly formed camp,recently made encampment,newly formed camp is released from the membran...,recently made encampment is released from the ...,0,0
1,one data,a particular statistic,"According to one data, in 1910, on others – in...","According to a particular statistic, in 1910, ...",1,1
2,particular structure,specific edifice,Note that Fact 1 does not assume any particula...,Note that Fact 1 does not assume any specific ...,0,2
3,involved people,participating individuals,Assessment-Center are usually group-processes ...,Assessment-Center are usually group-processes ...,1,3
4,different cross,opposing inquiries,"At the end of the 1980s, a different cross had...","At the end of the 1980s, a opposing inquiries ...",0,4
...,...,...,...,...,...,...
6999,similar notice,comparable notification,Nereus and Achilleus a similar notice was admi...,Nereus and Achilleus a comparable notification...,1,6999
7000,color map,painted chart,Color quantization is the process of creating ...,Color quantization is the process of creating ...,0,7000
7001,dutch tool,device from the people of the Netherlands,petra is a dutch tool with which patients and ...,petra is a device from the people of the Nethe...,1,7001
7002,secondary concern,less serious issue,"However, granting this access, especially to a...","However, granting this access, especially to a...",1,7002


In [4]:
import spacy
!python -m spacy download en_core_web_lg

nlp = spacy.load("en_core_web_lg")
doc = nlp("Assessment-Center are usually group-processes with high validity and acceptance of the involved people.")
for token in doc:
    print(token.text, token.pos_, token.dep_)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.3 MB/s eta 0:00:00m eta 0:00:010:00:02
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Assessment NOUN compound
- PUNCT punct
Center PROPN nsubj
are AUX ROOT
usually ADV advmod
group NOUN compound
- PUNCT punct
processes NOUN attr
with ADP prep
high ADJ amod
validity NOUN pobj
and CCONJ cc
acceptance NOUN conj
of ADP prep
the DET det
involved ADJ amod
people NOUN pobj
. PUNCT punct


In [5]:
import nltk
nltk.download('brown')

from nltk.corpus import brown    # Consider changing corpus (more tokens, etc.)

[nltk_data] Downloading package brown to /home/hari/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [6]:
import gensim as gs
from gensim.models import FastText

md = FastText(brown.sents(), min_count=1, vector_size=100, window=5)

In [7]:
p1 = 'former charge'
p2 = 'previous responsibility'

def avg_sim(p1, p2):
  p1_nlp = nlp(p1)
  p2_nlp = nlp(p2)


  avg_similarity = 0.0
  for t1 in p1_nlp:
    max_similarity = 0.0
    if t1.tag_ in ['PUNCT']:
      continue
    for t2 in p2_nlp:
      if t2.tag_ in ['PUNCT']:
        continue
      sim = md.wv.similarity(t1.text, t2.text)
      if sim > max_similarity:
        max_similarity = sim
    avg_similarity += max_similarity
    # print(avg_similarity)

  avg_similarity /= len([token for token in p1_nlp if token.tag_ != 'PUNCT'])
  if avg_similarity > 1:
    return 1

  return avg_similarity

In [8]:
avg_sim(p1, p2)

0.8866112232208252

In [9]:
avg_sim('smooth one', 'polished item')

0.6802492141723633

In [10]:
avg_sim('blue bag', 'blue ranch flavored bag')

1.0

In [14]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /home/hari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/hari/nltk_data...


In [15]:
def content_words_similarity(phrase, sentence):
  phrase_nlp = nlp(phrase)
  sent_nlp = nlp(sentence)

  root_phrase = phrase.split()[-1]

  content_words = []
  for token in sent_nlp:
    if token.tag_ in ('NN', 'NNS', 'NNP', 'NNPS'):
      content_words.append(token.text)

  # print(content_words)
  ents = []
  for ent in sent_nlp.ents:
    ents.append(ent.root.text)

  content_words = [word for word in content_words if word not in ents and word != root_phrase]

  avg_similarity = None
  for synset1 in wn.synsets(root_phrase):
    avg_similarity = 0.0
    for word in content_words:
      word_similarity = 0.0
      for synset2 in wn.synsets(word):
        sim = synset1.wup_similarity(synset2)
        if sim > word_similarity:
          word_similarity = sim
      avg_similarity += word_similarity
      # print(avg_similarity)

  if avg_similarity is None:
    return None

  try:
    avg_similarity /= len(content_words)
  except ZeroDivisionError:
    return None
  # print(content_words)

  return avg_similarity

In [16]:
content_words_similarity('one ensemble', 'Since 2004, the music department has taken one ensemble on a short UK tour in the summer term.')

0.26904761904761904

In [17]:
content_words_similarity('a group', 'Since 2004, the music department has taken a group on a short UK tour in the summer term.')

0.18672438672438674

In [18]:
train.shape

(7004, 6)

In [19]:
train_data = []
for _, row in train.head(7000).iterrows():
  train_data.append([avg_sim(row.phrase1, row.phrase2),
                     content_words_similarity(row.phrase1, row.sentence1),
                     content_words_similarity(row.phrase2, row.sentence2),
                     row.label])

test_data = []
for _, row in test.head(1000).iterrows():
  test_data.append([avg_sim(row.phrase1, row.phrase2),
                     content_words_similarity(row.phrase1, row.sentence1),
                     content_words_similarity(row.phrase2, row.sentence2),
                    row.label])



In [20]:
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

train_data = train_data.dropna()
test_data = test_data.dropna()

X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values

X_test = test_data.iloc[:, :-1].values
y_test = test_data.iloc[:, -1].values

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score

model = LogisticRegression()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

confusion_matrix(y_pred, y_test)

array([[269, 266],
       [218, 222]])

In [24]:
y_test

array([0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,

In [25]:
y_pred

array([1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,

In [26]:
X_train.shape

(6845, 3)

In [27]:
X_test.shape

(975, 3)

In [28]:
precision_score(y_pred, y_test)

0.45491803278688525

In [29]:
recall_score(y_pred, y_test)

0.5045454545454545

In [30]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_test)

0.5035897435897436

In [31]:
from sklearn.svm import SVC

model = SVC()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

confusion_matrix(y_pred, y_test)

array([[401, 366],
       [ 86, 122]])

In [32]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_test)

0.5364102564102564

In [33]:
pd.DataFrame(zip(y_test, y_pred))

,0,1
0,0,0
1,0,0
2,1,0
3,0,0
4,0,0
...,...,...
970,0,0
971,0,0
972,1,0
973,1,0


In [34]:
list(y_test).count(0)

487

In [35]:
val_data = []
for _, row in val.head(2000).iterrows():
  val_data.append([avg_sim(row.phrase1, row.phrase2),
                     content_words_similarity(row.phrase1, row.sentence1),
                     content_words_similarity(row.phrase2, row.sentence2),
                    row.label])

val_data = pd.DataFrame(val_data)

val_data = val_data.dropna()

X_val = val_data.iloc[:, :-1].values
y_val = val_data.iloc[:, -1].values

In [36]:
X_val

array([[0.81324235, 0.17249417, 0.32492997],
       [0.96105299, 0.29444444, 0.30707283],
       [0.88428345, 0.18492618, 0.51470588],
       ...,
       [0.94342157, 0.40357143, 0.61912519],
       [0.94793642, 0.16233766, 0.35      ],
       [0.74468341, 0.18636364, 0.17189523]])

In [37]:
val_data.describe()

,0,1,2,3
count,972.000000,972.000000,972.000000,972.000000
mean,0.857349,0.263864,0.285261,0.502058
std,0.078738,0.110261,0.115464,0.500253
min,0.366946,0.000000,0.000000,0.000000
25%,0.816156,0.192216,0.200162,0.000000
50%,0.870231,0.240823,0.267253,1.000000
75%,0.917461,0.311957,0.346151,1.000000
max,0.993685,1.000000,0.933333,1.000000


In [38]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

In [39]:
confusion_matrix(y_pred, y_test)

array([[223, 284],
       [264, 204]])

In [40]:
accuracy_score(y_pred, y_test)

0.4379487179487179

In [41]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

2024-02-15 07:44:31.531970: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-15 07:44:31.587903: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-15 07:44:31.865034: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 07:44:31.865175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 07:44:31.919652: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Epoch 1/100
107/107 [==============================] - 1s 2ms/step - loss: 0.6931 - accuracy: 0.4999 - val_loss: 0.6944 - val_accuracy: 0.5010
Epoch 2/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6930 - accuracy: 0.5096 - val_loss: 0.6938 - val_accuracy: 0.5031
Epoch 3/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6929 - accuracy: 0.5112 - val_loss: 0.6942 - val_accuracy: 0.5021
Epoch 4/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6934 - accuracy: 0.5043 - val_loss: 0.6937 - val_accuracy: 0.5041
Epoch 5/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6929 - accuracy: 0.5156 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 6/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6928 - accuracy: 0.5110 - val_loss: 0.6935 - val_accuracy: 0.5010
Epoch 7/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6932 - accuracy: 0.5064 - val_loss: 0.6935 - val_accuracy: 0.5113

Epoch 58/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6921 - accuracy: 0.5166 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 59/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6923 - accuracy: 0.5141 - val_loss: 0.6937 - val_accuracy: 0.5165
Epoch 60/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6923 - accuracy: 0.5123 - val_loss: 0.6939 - val_accuracy: 0.5051
Epoch 61/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6924 - accuracy: 0.5085 - val_loss: 0.6934 - val_accuracy: 0.5082
Epoch 62/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6921 - accuracy: 0.5112 - val_loss: 0.6952 - val_accuracy: 0.5010
Epoch 63/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6922 - accuracy: 0.5167 - val_loss: 0.6936 - val_accuracy: 0.5031
Epoch 64/100
107/107 [==============================] - 0s 1ms/step - loss: 0.6923 - accuracy: 0.5150 - val_loss: 0.6939 - val_accuracy:

In [42]:
model.evaluate(X_test, y_test)

31/31 [==============================] - 0s 697us/step - loss: 0.6918 - accuracy: 0.5005


[0.6918063163757324, 0.5005128383636475]

In [43]:
y_pred = model.predict(X_test)

y_pred = [1 if i>0.5 else 0 for [i] in y_pred]

31/31 [==============================] - 0s 739us/step


In [44]:
THRESHOLD = 0.5

y_pred = model.predict(X_test)
y_pred = [1 if i>THRESHOLD else 0 for [i] in y_pred]

confusion_matrix(y_pred, y_test)

31/31 [==============================] - 0s 545us/step


array([[293, 293],
       [194, 195]])

In [45]:
accuracy_score(y_pred, y_test)

0.5005128205128205

# Unconstrained

Same features, except use similarity from `all-MiniLM-L6-v2` transformer instead of w2v word similarity, etc

In [54]:
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer

bert = SentenceTransformer('all-MiniLM-L6-v2')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

/home/hari/anaconda3/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [55]:
import numpy as np

def similarity(w1, w2):
  v1, v2 = tuple(bert.encode([w1, w2]))
  return np.dot(v1, v2) / (np.dot(v1, v1) * np.dot(v2, v2)) ** 0.5

def avg_sim(p1, p2):
  p1_nlp = nlp(p1)
  p2_nlp = nlp(p2)


  avg_similarity = 0.0
  for t1 in p1_nlp:
    max_similarity = 0.0
    if t1.tag_ in ['PUNCT']:
      continue
    for t2 in p2_nlp:
      if t2.tag_ in ['PUNCT']:
        continue
      sim = similarity(t1.text, t2.text)
      if sim > max_similarity:
        max_similarity = sim
    avg_similarity += max_similarity
    # print(avg_similarity)

  avg_similarity /= len([token for token in p1_nlp if token.tag_ != 'PUNCT'])
  if avg_similarity > 1:
    return 1

  return avg_similarity

In [56]:
similarity('good', 'bad')

0.587149346640746

In [57]:
def content_words_similarity(phrase, sentence):
  phrase_nlp = nlp(phrase)
  sent_nlp = nlp(sentence)

  root_phrase = phrase.split()[-1]

  content_words = []
  for token in sent_nlp:
    if token.tag_ in ('NN', 'NNS', 'NNP', 'NNPS'):
      content_words.append(token.text)

  # print(content_words)
  ents = []
  for ent in sent_nlp.ents:
    ents.append(ent.root.text)

  content_words = [word for word in content_words if word not in ents and word != root_phrase]

  sim = 1
  for word in content_words:
    sim *= (similarity(root_phrase, word))

  if sim == 1:
    return None

  return sim

In [58]:
train_data = []
for _, row in train.head(7000).iterrows():
  train_data.append([avg_sim(row.phrase1, row.phrase2),
                     content_words_similarity(row.phrase1, row.sentence1),
                     content_words_similarity(row.phrase2, row.sentence2),
                     row.label])

test_data = []
for _, row in test.head(1000).iterrows():
  test_data.append([avg_sim(row.phrase1, row.phrase2),
                     content_words_similarity(row.phrase1, row.sentence1),
                     content_words_similarity(row.phrase2, row.sentence2),
                    row.label])



In [59]:
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

train_data = train_data.dropna()
test_data = test_data.dropna()

X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values

X_test = test_data.iloc[:, :-1].values
y_test = test_data.iloc[:, -1].values

In [60]:
model = LogisticRegression()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

confusion_matrix(y_pred, y_test)

array([[310, 281],
       [189, 216]])

In [61]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_test)

0.5281124497991968

In [62]:
from sklearn.svm import SVC

model = SVC()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

confusion_matrix(y_pred, y_test)

array([[435, 434],
       [ 64,  63]])

In [63]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_test)

0.5

In [64]:
val_data = []
for _, row in val.head(2000).iterrows():
  val_data.append([avg_sim(row.phrase1, row.phrase2),
                     content_words_similarity(row.phrase1, row.sentence1),
                     content_words_similarity(row.phrase2, row.sentence2),
                    row.label])

val_data = pd.DataFrame(val_data)

val_data = val_data.dropna()

X_val = val_data.iloc[:, :-1].values
y_val = val_data.iloc[:, -1].values

In [72]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

In [73]:
confusion_matrix(y_pred, y_test)

array([[265, 255],
       [234, 242]])

In [74]:
accuracy_score(y_pred, y_test)

0.5090361445783133

In [67]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/100
109/109 [==============================] - 1s 2ms/step - loss: 0.6927 - accuracy: 0.5106 - val_loss: 0.6931 - val_accuracy: 0.4995
Epoch 2/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6932 - accuracy: 0.4976 - val_loss: 0.6913 - val_accuracy: 0.5559
Epoch 3/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6920 - accuracy: 0.5130 - val_loss: 0.6905 - val_accuracy: 0.5347
Epoch 4/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6924 - accuracy: 0.5091 - val_loss: 0.6896 - val_accuracy: 0.5690
Epoch 5/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6913 - accuracy: 0.5238 - val_loss: 0.6886 - val_accuracy: 0.5650
Epoch 6/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6912 - accuracy: 0.5215 - val_loss: 0.6901 - val_accuracy: 0.5045
Epoch 7/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6910 - accuracy: 0.5217 - val_loss: 0.6882 - val_accuracy: 0.5680

Epoch 58/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6848 - accuracy: 0.5399 - val_loss: 0.6816 - val_accuracy: 0.5438
Epoch 59/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6845 - accuracy: 0.5402 - val_loss: 0.6808 - val_accuracy: 0.5498
Epoch 60/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6851 - accuracy: 0.5365 - val_loss: 0.6811 - val_accuracy: 0.5509
Epoch 61/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6860 - accuracy: 0.5306 - val_loss: 0.6806 - val_accuracy: 0.5509
Epoch 62/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6851 - accuracy: 0.5372 - val_loss: 0.6810 - val_accuracy: 0.5478
Epoch 63/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6858 - accuracy: 0.5369 - val_loss: 0.6802 - val_accuracy: 0.5408
Epoch 64/100
109/109 [==============================] - 0s 1ms/step - loss: 0.6850 - accuracy: 0.5404 - val_loss: 0.6839 - val_accuracy:

In [68]:
model.evaluate(X_test, y_test)

32/32 [==============================] - 0s 672us/step - loss: 0.6910 - accuracy: 0.5402


[0.691016435623169, 0.5401606559753418]

In [69]:
y_pred = model.predict(X_test)

y_pred = [1 if i>0.5 else 0 for [i] in y_pred]

32/32 [==============================] - 0s 647us/step


In [70]:
THRESHOLD = 0.5

y_pred = model.predict(X_test)
y_pred = [1 if i>THRESHOLD else 0 for [i] in y_pred]

confusion_matrix(y_pred, y_test)

32/32 [==============================] - 0s 709us/step


array([[350, 309],
       [149, 188]])

In [71]:
accuracy_score(y_pred, y_test)

0.5401606425702812